# Python Project for Data Engineering

## Project Overview

For this project, you will assume the role of data engineer working for an international financial analysis company. Your company tracks stock prices, commodities, forex rates, inflation rates.  Your job is to extract financial data from various sources like websites, APIs and files provided by various financial analysis firms. After you collect the data, you extract the data of interest to your company and transform it based on the requirements given to you. Once the transformation is complete you load that data into a database.

## Project Tasks

* Collect data using APIs

* Collect data using webscraping.

* Download files to process.    

* Read csv, xml and json file types.

* Extract data from the above file types.

* Transform data.

* Use the built in logging module.

* Save the transformed data in a ready-to-load format which data engineers can use to load the data.



## Extract, Transform, Load

In [42]:
import glob                         # this module helps in selecting files 
import pandas as pd
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

### Download Files

In [43]:
import zipfile
with zipfile.ZipFile(r"C:\Users\baiba\Downloads\source.zip") as zip_ref:
    zip_ref.extractall("targetdir")

In [44]:
tmpfile    = "temp.tmp"               # file used to store all extracted data
logfile    = "logfile.txt"            # all event logs will be stored in this file
targetfile = "transformed_data.csv"   # file where transformed data is stored

## Extract

### CSV Extract Function

In [45]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

### JSON Extract Function

In [25]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

### XML Extract Function

In [26]:
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["name", "height", "weight"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)
        dataframe = dataframe.append({"name":name, "height":height, "weight":weight}, ignore_index=True)
    return dataframe

## Exctract Function
 

In [46]:
def extract():
    extracted_data = pd.DataFrame(columns=['name','height','weight']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

## Transform

In [28]:
def transform(data):
        #Convert height which is in inches to millimeter
        #Convert the datatype of the column into float
        #data.height = data.height.astype(float)
        #Convert inches to meters and round off to two decimals(one inch is 0.0254 meters)
        data['height'] = round(data.height * 0.0254,2)
        
        #Convert weight which is in pounds to kilograms
        #Convert the datatype of the column into float
        #data.weight = data.weight.astype(float)
        #Convert pounds to kilograms and round off to two decimals(one pound is 0.45359237 kilograms)
        data['weight'] = round(data.weight * 0.45359237,2)
        return data

## Loading

In [29]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile) 

## Logging

In [47]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running the ETL Process

In [48]:
log("ETL Job Started")

In [ ]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

In [ ]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 

In [ ]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

# Webscraping

The wikipedia webpage https://en.wikipedia.org/wiki/List_of_largest_banks provides information about largest banks in the world by various parameters. Scrape the data from the table 'By market capitalization' and store it in a JSON file.

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [4]:
html_data = requests.get("https://en.wikipedia.org/wiki/List_of_largest_banks").text

In [8]:
html_data[101:124]

'List of largest banks -'

In [9]:
soup = BeautifulSoup(html_data, "html.parser")

In [10]:
data = pd.DataFrame(columns=["Name", "Market Cap (US$ Billion)"])

Load the data from the By market capitalization table into a pandas dataframe. The dataframe should have the country Name and Market Cap (US$ Billion) as column names. Using the empty dataframe data and the given loop extract the necessary data from each row and append it to the empty dataframe.

In [11]:
for row in soup.find_all('tbody')[3].find_all('tr'):
    col = row.find_all('td')
    if col:
        bank_name = col[1].find_all("a")[1].text
        market_cap = float(col[2].text)
        data = data.append({"Name": bank_name,
                            "Market Cap (US$ Billion)": market_cap},
                           ignore_index=True)

In [12]:
data.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,368.78
1,Industrial and Commercial Bank of China,295.65
2,Bank of America,279.73
3,Wells Fargo,214.34
4,China Construction Bank,207.98


In [13]:
data.to_json(r'largest_bank.json') #export to json

# Extract API Data

In [15]:
url = "http://api.exchangeratesapi.io/v1/latest?base=EUR&access_key=3ae8a41ae8b314fe2c76abee00969ef2"  
response = requests.get(url)
response.json()

{'success': True,
 'timestamp': 1647945844,
 'base': 'EUR',
 'date': '2022-03-22',
 'rates': {'AED': 4.03948,
  'AFN': 97.31795,
  'ALL': 122.850425,
  'AMD': 539.199454,
  'ANG': 1.988467,
  'AOA': 501.494935,
  'ARS': 120.948615,
  'AUD': 1.48099,
  'AWG': 1.980159,
  'AZN': 1.870276,
  'BAM': 1.956173,
  'BBD': 2.22765,
  'BDT': 94.913444,
  'BGN': 1.955764,
  'BHD': 0.414578,
  'BIF': 2262.687347,
  'BMD': 1.099783,
  'BND': 1.496576,
  'BOB': 7.596501,
  'BRL': 5.428194,
  'BSD': 1.103268,
  'BTC': 2.5700761e-05,
  'BTN': 83.917278,
  'BWP': 12.76287,
  'BYN': 3.606267,
  'BYR': 21555.743907,
  'BZD': 2.22395,
  'CAD': 1.38406,
  'CDF': 2206.16429,
  'CHF': 1.029441,
  'CLF': 0.031637,
  'CLP': 872.963798,
  'CNY': 6.998692,
  'COP': 4185.223645,
  'CRC': 711.081157,
  'CUC': 1.099783,
  'CUP': 29.144246,
  'CVE': 110.285342,
  'CZK': 24.729171,
  'DJF': 196.41087,
  'DKK': 7.439613,
  'DOP': 60.529164,
  'DZD': 156.665859,
  'EGP': 20.348951,
  'ERN': 16.496748,
  'ETB': 56.63474

### Save as DataFrame

In [16]:
dataframe = pd.DataFrame(response.json())
dataframe

,success,timestamp,base,date,rates
AED,True,1647945844,EUR,2022-03-22,4.039480
AFN,True,1647945844,EUR,2022-03-22,97.317950
ALL,True,1647945844,EUR,2022-03-22,122.850425
AMD,True,1647945844,EUR,2022-03-22,539.199454
ANG,True,1647945844,EUR,2022-03-22,1.988467
...,...,...,...,...,...
YER,True,1647945844,EUR,2022-03-22,275.220858
ZAR,True,1647945844,EUR,2022-03-22,16.357796
ZMK,True,1647945844,EUR,2022-03-22,9899.365709
ZMW,True,1647945844,EUR,2022-03-22,19.528469


In [17]:
#Drop unnecessary columns
dataframe.drop(columns=["success", "timestamp", "base", "date"], inplace=True)
dataframe

,rates
AED,4.039480
AFN,97.317950
ALL,122.850425
AMD,539.199454
ANG,1.988467
...,...
YER,275.220858
ZAR,16.357796
ZMK,9899.365709
ZMW,19.528469


### Load the data

In [18]:
dataframe.to_csv("exchange_rates_1.csv")